In [1]:
import httplib2
import argparse
import pandas as pd
import requests
import json
from pprint import pprint
from datetime import datetime, timedelta
from yaml import load, FullLoader
from pprint import pprint
from oauth2client.file import Storage
from oauth2client import tools
from oauth2client import client
from googleapiclient import discovery

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CLIENT_SECRET_FILE = 'client_secret_grants.json' 
APPLICATION_NAME = 'Google Sheets API Report'
credential_path = 'sheets.googleapis.com-report.json'

In [3]:
store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])
flags = parser.parse_args([])
if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
print('Storing credentials to ' + credential_path)

Storing credentials to sheets.googleapis.com-report.json


In [4]:
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl = discoveryUrl)
spreadsheetId = '1CmQqTVuzqDs5WX7fNAlB43AvTsoOfYqIE266impvCJI'

In [5]:
rangeName = 'partners dict!A1:C10'

In [6]:
result = service.spreadsheets().values().get(spreadsheetId = spreadsheetId, range = rangeName).execute()
values = result.get('values', [])

In [7]:
pprint(values)

[['Партнер', 'utm_source', 'utm_medium'],
 ['СМИ2', 'smi2', 'cpm'],
 ['РБК', 'rbc', 'cpc'],
 ['Infox', 'infox', 'cpc'],
 ['Infox', 'infox-action', 'cpa']]


In [8]:
headers = values[0]
rows = values[1:]
data_top = pd.DataFrame.from_records(rows, columns = headers)
data_bottom = pd.read_csv('data/metrika_data.txt', sep = '\t')

In [9]:
data_top.head()

,Партнер,utm_source,utm_medium
0,СМИ2,smi2,cpm
1,РБК,rbc,cpc
2,Infox,infox,cpc
3,Infox,infox-action,cpa


In [10]:
data_bottom.head()

,utm_source,utm_medium,sales
0,infox,cpc,1534
1,smi2,cpm,1495
2,infox-action,cpa,735
3,rbc,cpc,59


In [11]:
result = pd.merge(data_top, data_bottom, how='outer', on=['utm_source', 'utm_medium'])

In [12]:
result.head()

,Партнер,utm_source,utm_medium,sales
0,СМИ2,smi2,cpm,1495
1,РБК,rbc,cpc,59
2,Infox,infox,cpc,1534
3,Infox,infox-action,cpa,735


## Параметры Яндекс метрики

In [13]:
config_file = open('config.yaml', 'r')
config = load(config_file, Loader=FullLoader)
token = config['token']
headers = {'Authorization': 'OAuth ' + token}
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
counter = '51583145'
startDate = '2020-10-01'
endDate = '2021-01-31'
dimensions = ['ym:s:date', 'ym:s:deviceCategory', 'ym:s:hasAdBlocker', 'ym:s:trafficSource']
metrics = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews']
metrics_string = ','.join(metrics)
dimensions_string = ','.join(dimensions)

In [14]:
f = open('data/report_params.yaml', 'r')
params = load(f, Loader=FullLoader)
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
# startDate = (datetime.now() - timedelta(days = params['reports']['dau']['days_ago'])).strftime('%Y-%m-%d')
# endDate = (datetime.now() - timedelta(days = 1)).strftime('%Y-%m-%d')
dimensions = params['reports']['dau']['dimensions']
metrics = params['reports']['dau']['metrics']
sortby = params['reports']['dau']['sort']
counter = params['metrika counter']

In [15]:
def make_request(**kwargs):
    """Запрос к API Яндекс.Метрики. Возвращает JSON ответа"""
    return requests.get(API_URL, headers=headers, params = kwargs).json()

In [16]:
def json_handler(data):
    """Преобразование ответа JSON в спикок листов"""
    report = []
    for line in data['data']:
        dimension_list = [x['name'] for x in line['dimensions']]
        metrics_list = line['metrics']
        report.append(dimension_list + metrics_list)
    return report

In [17]:
response = make_request(date1 = startDate, date2 = endDate, dimensions = dimensions, metrics = metrics, 
                    id = counter, sort = sortby, accuracy = 1)

In [24]:
data = json_handler(response)
data

2021-01-31


[['2020-10-01', 100.0, 89.0, 232.0, 14.0],
 ['2020-10-02', 79.0, 73.0, 178.0, 20.25316456],
 ['2020-10-03', 54.0, 51.0, 107.0, 12.96296296],
 ['2020-10-04', 68.0, 60.0, 123.0, 20.58823529],
 ['2020-10-05', 91.0, 80.0, 211.0, 19.78021978],
 ['2020-10-06', 105.0, 92.0, 208.0, 19.04761905],
 ['2020-10-07', 60.0, 58.0, 130.0, 21.66666667],
 ['2020-10-08', 86.0, 80.0, 165.0, 22.09302326],
 ['2020-10-09', 80.0, 75.0, 161.0, 18.75],
 ['2020-10-10', 68.0, 65.0, 161.0, 19.11764706],
 ['2020-10-11', 78.0, 74.0, 142.0, 23.07692308],
 ['2020-10-12', 82.0, 80.0, 196.0, 25.6097561],
 ['2020-10-13', 88.0, 80.0, 170.0, 13.63636364],
 ['2020-10-14', 64.0, 57.0, 116.0, 18.75],
 ['2020-10-15', 87.0, 79.0, 166.0, 25.28735632],
 ['2020-10-16', 86.0, 80.0, 186.0, 24.41860465],
 ['2020-10-17', 47.0, 42.0, 98.0, 23.40425532],
 ['2020-10-18', 53.0, 50.0, 107.0, 16.98113208],
 ['2020-10-19', 96.0, 88.0, 195.0, 21.875],
 ['2020-10-20', 86.0, 80.0, 211.0, 24.41860465],
 ['2020-10-21', 68.0, 65.0, 151.0, 23.529411

In [19]:
range_data = params['reports']['dau']['range_name']
range_name = '{}!{}{}:{}{}'.format(range_data['list'], range_data['start'], 2, range_data['end'], len(data) + 1)

In [20]:
range_name

'daily_metrics!A2:E101'

In [21]:
body = {'values': data}

In [22]:
pprint(body)

{'values': [['2020-10-01', 100.0, 89.0, 232.0, 14.0],
            ['2020-10-02', 79.0, 73.0, 178.0, 20.25316456],
            ['2020-10-03', 54.0, 51.0, 107.0, 12.96296296],
            ['2020-10-04', 68.0, 60.0, 123.0, 20.58823529],
            ['2020-10-05', 91.0, 80.0, 211.0, 19.78021978],
            ['2020-10-06', 105.0, 92.0, 208.0, 19.04761905],
            ['2020-10-07', 60.0, 58.0, 130.0, 21.66666667],
            ['2020-10-08', 86.0, 80.0, 165.0, 22.09302326],
            ['2020-10-09', 80.0, 75.0, 161.0, 18.75],
            ['2020-10-10', 68.0, 65.0, 161.0, 19.11764706],
            ['2020-10-11', 78.0, 74.0, 142.0, 23.07692308],
            ['2020-10-12', 82.0, 80.0, 196.0, 25.6097561],
            ['2020-10-13', 88.0, 80.0, 170.0, 13.63636364],
            ['2020-10-14', 64.0, 57.0, 116.0, 18.75],
            ['2020-10-15', 87.0, 79.0, 166.0, 25.28735632],
            ['2020-10-16', 86.0, 80.0, 186.0, 24.41860465],
            ['2020-10-17', 47.0, 42.0, 98.0, 23.40425532],

In [23]:
service.spreadsheets().values().update(spreadsheetId=spreadsheetId, range=range_name, valueInputOption='USER_ENTERED', body=body).execute()

{'spreadsheetId': '1CmQqTVuzqDs5WX7fNAlB43AvTsoOfYqIE266impvCJI',
 'updatedRange': 'daily_metrics!A2:E101',
 'updatedRows': 100,
 'updatedColumns': 5,
 'updatedCells': 500}